In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# drive/My Drive/datasets/
test = open("drive/My Drive/datasets/idk.txt","w")
test.write("this is a test")
test.close()

In [ ]:
!unzip drive/MyDrive/MyDrive/irv22.zip/irv23.zip


Archive:  drive/MyDrive/MyDrive/irv22.zip/irv23.zip
   creating: irv22/
  inflating: irv22/X_train_irv22.npy  
  inflating: irv22/X_val_irv22.npy   
  inflating: irv22/y_train_e_irv22.npy  
  inflating: irv22/y_val_e_irv22.npy  


In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
X_train = np.load("irv22/X_train_irv22.npy")
print(X_train.shape)
X_val = np.load("irv22/X_val_irv22.npy")
y_train_e = np.load("irv22/y_train_e_irv22.npy")
y_val_e = np.load("irv22/y_val_e_irv22.npy" )

(3807, 299, 299, 3)


In [ ]:
from keras.layers import Input, Conv2D, Lambda, MaxPool2D, UpSampling2D, AveragePooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers import Activation, Flatten, Dense, Add, Multiply, BatchNormalization, Dropout, concatenate
from keras.models import Model
from keras import backend as K
class IRV2():
    def __init__(self, input_shape, n_classes, activation, scale1, scale2, scale3, p=1, t=2, r=1):
        self.input_shape = input_shape
        self.n_classes = n_classes
        self.activation = activation
        self.test_scales = [0.1, 0.15, 0.2, 0.25, 0.3]
        self.scale1 = scale1
        self.scale2 = scale2
        self.scale3 = scale3

    def build_model(self):
        input_data = Input(shape=self.input_shape)
        stem = self.stem(input_data)
        ira = Activation("relu")(stem)
        for i in range(5):
            ira = self.IRA(ira)

        ra = self.RA(ira)

        irb = Activation("relu")(ra)
        for i in range(10):
            irb = self.IRB(irb)

        rb = self.RB(irb)

        irc = Activation("relu")(rb)
        for i in range(5):
            irc = self.IRC(irc)

        pool = GlobalAveragePooling2D()(irc)

        drop = Dropout(0.2)(pool)
        final = Dense(self.n_classes, activation= self.activation)(drop)
        model = Model(inputs = input_data, outputs = final)

        return model


    def stem(self, input_data):
        conv_1 = Conv2D(32, (3,3), strides=2, padding="valid")(input_data)
        conv_1 = BatchNormalization()(conv_1)
        conv_1 = Activation('relu')(conv_1)

        conv_2 = Conv2D(32, (3, 3), padding="valid")(conv_1)
        conv_2 = BatchNormalization()(conv_2)
        conv_2 = Activation('relu')(conv_2)

        conv_3 = Conv2D(64, (3, 3), padding="same")(conv_2)
        conv_3 = BatchNormalization()(conv_3)
        conv_3 = Activation('relu')(conv_3)

        pool_1 = MaxPool2D((3,3), strides=2, padding='valid')(conv_3)

        conv_4 = Conv2D(96, (3, 3), strides = 2, padding="valid")(conv_3)
        conv_4 = BatchNormalization()(conv_4)
        conv_4 = Activation('relu')(conv_4)

        conc_1 = concatenate([pool_1, conv_4])

        conv_5_1 = Conv2D(64, (1, 1), padding="same")(conc_1)
        conv_5_1 = BatchNormalization()(conv_5_1)
        conv_5_1 = Activation('relu')(conv_5_1)

        conv_6_1 = Conv2D(96, (3, 3), padding="valid")(conv_5_1)
        conv_6_1 = BatchNormalization()(conv_6_1)
        conv_6_1 = Activation('relu')(conv_6_1)

        conv_5_2 = Conv2D(64, (1, 1), padding="same")(conc_1)
        conv_5_2 = BatchNormalization()(conv_5_2)
        conv_5_2 = Activation('relu')(conv_5_2)

        conv_6_2 = Conv2D(64, (7, 1), padding="same")(conv_5_2)
        conv_6_2 = BatchNormalization()(conv_6_2)
        conv_6_2 = Activation('relu')(conv_6_2)

        conv_7_2 = Conv2D(64, (1, 7), padding="same")(conv_6_2)
        conv_7_2 = BatchNormalization()(conv_7_2)
        conv_7_2 = Activation('relu')(conv_7_2)

        conv_8_2 = Conv2D(96, (3, 3), padding="valid")(conv_7_2)
        conv_8_2 = BatchNormalization()(conv_8_2)
        conv_8_2 = Activation('relu')(conv_8_2)

        conc_2 = concatenate([conv_6_1, conv_8_2])

        conv_1_3 = Conv2D(192, (3,3),strides=2, padding="valid")(conc_2)
        conv_1_3 = BatchNormalization()(conv_1_3)
        conv_1_3 = Activation("relu")(conv_1_3)

        pool_1_4 = MaxPool2D(strides=2, padding = "valid")(conc_2)

        conc_3 = concatenate([conv_1_3, pool_1_4])

        return conc_3

    #fig 16
    def IRA(self, input_data):
        # relu_1 = Activation("relu")(input_data)
        b_0 = Conv2D(32, (1,1), padding="same")(input_data)
        b_0 = BatchNormalization()(b_0)
        b_0 = Activation("relu")(b_0)

        b_1 = Conv2D(32, (1, 1), padding="same")(input_data)
        b_1 = BatchNormalization()(b_1)
        b_1 = Activation("relu")(b_1)
        b_1 = Conv2D(32, (3, 3), padding="same")(b_1)
        b_1 = BatchNormalization()(b_1)
        b_1 = Activation("relu")(b_1)

        b_2 = Conv2D(32, (1, 1), padding="same")(input_data)
        b_2 = BatchNormalization()(b_2)
        b_2 = Activation("relu")(b_2)
        b_2 = Conv2D(48, (3, 3), padding="same")(b_2)
        b_2 = BatchNormalization()(b_2)
        b_2 = Activation("relu")(b_2)
        b_2 = Conv2D(64, (3, 3), padding="same")(b_2)
        b_2 = BatchNormalization()(b_2)
        b_2 = Activation("relu")(b_2)

        conc_1 = concatenate([b_0, b_1, b_2])

        conv = Conv2D(384, (1,1), padding="same")(conc_1)

        out = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
               output_shape=K.int_shape(conv)[1:],
               arguments={'scale': self.scale1})([input_data, conv])
        out = Activation("relu")(out)

        return out

    # fig 7
    # k l m n = 256 256 384 384
    def RA(self, input_data):
        b_0 = MaxPool2D((3,3), strides=2, padding="valid")(input_data)

        b_1 = Conv2D(384, (3,3), strides=2, padding="valid")(input_data)
        b_1 = BatchNormalization()(b_1)
        b_1 = Activation("relu")(b_1)

        b_2 = Conv2D(256, (1,1), padding="same")(input_data)
        b_2 = BatchNormalization()(b_2)
        b_2 = Activation("relu")(b_2)
        b_2 = Conv2D(256, (3,3), padding="same")(b_2)
        b_2 = BatchNormalization()(b_2)
        b_2 = Activation("relu")(b_2)
        b_2 = Conv2D(384, (3,3), strides=2, padding="valid")(b_2)
        b_2 = BatchNormalization()(b_2)
        b_2 = Activation("relu")(b_2)

        conc_1 = concatenate([b_0, b_1, b_2])

        return conc_1

    # fig 17
    def IRB(self, input_data):
        b_1 = Conv2D(192, (1,1), padding="same")(input_data)
        b_1 = BatchNormalization()(b_1)
        b_1 = Activation("relu")(b_1)

        b_2 = Conv2D(128, (1,1), padding="same")(input_data)
        b_2 = BatchNormalization()(b_2)
        b_2 = Activation("relu")(b_2)
        b_2 = Conv2D(160, (1,7), padding="same")(b_2)
        b_2 = BatchNormalization()(b_2)
        b_2 = Activation("relu")(b_2)
        b_2 = Conv2D(192, (7,1), padding="same")(b_2)
        b_2 = BatchNormalization()(b_2)
        b_2 = Activation("relu")(b_2)

        conc_1 = concatenate([b_1, b_2])

        conv = Conv2D(1152, (1,1), padding="same")(conc_1)

        out = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
               output_shape=K.int_shape(conv)[1:],
               arguments={'scale': self.scale2})([input_data, conv])
        out = Activation("relu")(out)
        return out


    def RB(self, input_data):
        b_0 = MaxPool2D((3,3), strides=2, padding="valid")(input_data)

        b_1 = Conv2D(256, (1,1), padding="same")(input_data)
        b_1 = BatchNormalization()(b_1)
        b_1 = Activation("relu")(b_1)
        b_1 = Conv2D(384, (3,3),strides=2, padding="valid")(b_1)
        b_1 = BatchNormalization()(b_1)
        b_1 = Activation("relu")(b_1)

        b_2 = Conv2D(256, (1,1), padding="same")(input_data)
        b_2 = BatchNormalization()(b_2)
        b_2 = Activation("relu")(b_2)
        b_2 = Conv2D(288, (3,3),strides=2, padding="valid")(b_2)
        b_2 = BatchNormalization()(b_2)
        b_2 = Activation("relu")(b_2)

        b_3 = Conv2D(256, (1,1), padding="same")(input_data)
        b_3 = BatchNormalization()(b_3)
        b_3 = Activation("relu")(b_3)
        b_3 = Conv2D(288, (3,3), padding="same")(b_3)
        b_3 = BatchNormalization()(b_3)
        b_3 = Activation("relu")(b_3)
        b_3 = Conv2D(320, (3,3),strides=2, padding="valid")(b_3)
        b_3 = BatchNormalization()(b_3)
        b_3 = Activation("relu")(b_3)

        conc = concatenate([b_0,b_1,b_2,b_3])
        return conc

    def IRC(self, input_data):
        b_1 = Conv2D(192, (1,1), padding="same")(input_data)
        b_1 = BatchNormalization()(b_1)
        b_1 = Activation("relu")(b_1)

        b_2 = Conv2D(192, (1,1), padding="same")(input_data)
        b_2 = BatchNormalization()(b_2)
        b_2 = Activation("relu")(b_2)
        b_2 = Conv2D(224, (1,3), padding="same")(b_2)
        b_2 = BatchNormalization()(b_2)
        b_2 = Activation("relu")(b_2)
        b_2 = Conv2D(256, (3,1), padding="same")(b_2)
        b_2 = BatchNormalization()(b_2)
        b_2 = Activation("relu")(b_2)

        conc_1 = concatenate([b_1, b_2])

        conv = Conv2D(2144, (1,1), padding="same")(conc_1)
        out = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
               output_shape=K.int_shape(conv)[1:],
               arguments={'scale': self.scale3})([input_data, conv])
        relu = Activation("relu")(out)
        return relu

In [ ]:
#i, j, k = 0.2, 0.1, 0.25
i, j, k = 0.3, 0.25, 0.2
# tf.keras.backend.clear_session()
# tf.keras.backend.set_learning_phase(True)
model = IRV2(X_train.shape[1:], 48, activation= 'softmax', scale1=i, scale2=j, scale3=k).build_model()
model.compile(loss='sparse_categorical_crossentropy',optimizer="adam",
                metrics=['accuracy'])
epochs = 50
history = model.fit(X_train, y_train_e, batch_size=32, epochs=epochs, validation_data=(X_val, y_val_e))
model.save("IRV2_Final1")
tmp = pd.DataFrame.from_dict(history.history)
tmp.to_csv("IRV2_history.csv")
!cp -R IRV2_Final1/ drive/MyDrive/IRV2_Final1
!cp IRV2_history.csv drive/MyDrive/IRV2_history1.csv

Epoch 1/50
119/119 [==============================] - 146s 939ms/step - loss: 2.9512 - accuracy: 0.3202 - val_loss: 3.0532 - val_accuracy: 0.2569
Epoch 2/50
119/119 [==============================] - 103s 870ms/step - loss: 2.0997 - accuracy: 0.3961 - val_loss: 2.7914 - val_accuracy: 0.3270
Epoch 3/50
119/119 [==============================] - 105s 885ms/step - loss: 1.7695 - accuracy: 0.4870 - val_loss: 2.9577 - val_accuracy: 0.3759
Epoch 4/50
119/119 [==============================] - 106s 890ms/step - loss: 1.4621 - accuracy: 0.5737 - val_loss: 2.6836 - val_accuracy: 0.3696
Epoch 5/50
119/119 [==============================] - 106s 889ms/step - loss: 1.0962 - accuracy: 0.6835 - val_loss: 1.9533 - val_accuracy: 0.5051
Epoch 6/50
119/119 [==============================] - 106s 890ms/step - loss: 0.7851 - accuracy: 0.7757 - val_loss: 1.8672 - val_accuracy: 0.5768
Epoch 7/50
119/119 [==============================] - 106s 889ms/step - loss: 0.5725 - accuracy: 0.8293 - val_loss: 2.5705 -

In [ ]:
history1.history

{'accuracy': [0.9966844916343689,
  0.9977046847343445,
  0.9987248182296753,
  0.9982147216796875,
  0.9977046847343445],
 'loss': [0.014184854924678802,
  0.009055106900632381,
  0.006048876326531172,
  0.0038963521365076303,
  0.004437550902366638],
 'val_accuracy': [0.9571537971496582,
  0.9655700325965881,
  0.9686304330825806,
  0.9701606631278992,
  0.967100203037262],
 'val_loss': [0.17321275174617767,
  0.13784149289131165,
  0.13420701026916504,
  0.13053496181964874,
  0.12961040437221527]}

In [ ]:
tmp = pd.DataFrame.from_dict(history.history)
tmp.to_csv("IRV2_history.csv")
#!cp -R IRV2_Final1/ drive/MyDrive/IRV2_Final1
!cp IRV2_history.csv drive/MyDrive/IRV2_h.csv
tmp = pd.DataFrame.from_dict(history1.history)
tmp.to_csv("IRV2_history1.csv")
#!cp -R IRV2_Final1/ drive/MyDrive/IRV2_Final1
!cp IRV2_history.csv drive/MyDrive/IRV2_h1.csv

In [ ]:
epochs = 5
history1 = model.fit(X_train, y_train_e, batch_size=32, epochs=epochs, validation_data=(X_val, y_val_e))

Epoch 1/5
123/123 [==============================] - 104s 847ms/step - loss: 0.0142 - accuracy: 0.9967 - val_loss: 0.1732 - val_accuracy: 0.9572
Epoch 2/5
123/123 [==============================] - 106s 861ms/step - loss: 0.0091 - accuracy: 0.9977 - val_loss: 0.1378 - val_accuracy: 0.9656
Epoch 3/5
123/123 [==============================] - 108s 878ms/step - loss: 0.0060 - accuracy: 0.9987 - val_loss: 0.1342 - val_accuracy: 0.9686
Epoch 4/5
123/123 [==============================] - 109s 885ms/step - loss: 0.0039 - accuracy: 0.9982 - val_loss: 0.1305 - val_accuracy: 0.9702
Epoch 5/5
123/123 [==============================] - 109s 887ms/step - loss: 0.0044 - accuracy: 0.9977 - val_loss: 0.1296 - val_accuracy: 0.9671


In [ ]:
model.save("IRV2_Final1")
tmp = pd.DataFrame.from_dict(history1.history)
tmp.to_csv("IRV2_history2.csv")
!cp -R IRV2_Final1/ drive/MyDrive/IRV2_Final2
!cp IRV2_history1.csv drive/MyDrive/IRV2_history1.csv

KeyboardInterrupt: ignored

In [ ]:
def model_builder(hp):
    test_scales = [0.1, 0.15, 0.2, 0.25, 0.3]
    scale1 = hp.Choice("Scale 1", test_scales)
    scale2 = hp.Choice("Scale 2", test_scales)
    scale3 = hp.Choice("Scale 3", test_scales)
    model = IRV2((299,299,3), 48, activation= 'softmax', scale1=scale1, scale2=scale2, scale3=scale3).build_model()
    model.compile(loss='sparse_categorical_crossentropy',optimizer="adam",
                metrics=['accuracy'])
    return model

tuner = kt.Hyperband(
    hypermodel=model_builder,
    objective="val_accuracy",
    project_name="hband",
    directory="drive/My\ Drive/AlexNetStuff",
    hyperband_iterations=2,
    max_epochs=50
)

tuner.search(X_train, y_train_e, epochs=50, validation_data=(X_val, y_val_e), callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

In [ ]:
tuner = kt.Hyperband(
    hypermodel=model_builder,
    objective="val_accuracy",
    project_name="hband",
    directory="drive/My\ Drive/AlexNetStuff",
    hyperband_iterations=2,
    max_epochs=50
)

INFO:tensorflow:Reloading Oracle from existing project test/hband/oracle.json
INFO:tensorflow:Reloading Tuner from test/hband/tuner0.json


In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
Scale 1 (Choice)
{'default': 0.1, 'conditions': [], 'values': [0.1, 0.15, 0.2, 0.25, 0.3], 'ordered': True}
Scale 2 (Choice)
{'default': 0.1, 'conditions': [], 'values': [0.1, 0.15, 0.2, 0.25, 0.3], 'ordered': True}
Scale 3 (Choice)
{'default': 0.1, 'conditions': [], 'values': [0.1, 0.15, 0.2, 0.25, 0.3], 'ordered': True}


In [ ]:
tuner.search(X_train, y_train_e, epochs=50, validation_data=(X_val, y_val_e), callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])
tuner.results_summary()

Trial 83 Complete [00h 22m 50s]
val_accuracy: 0.8704883456230164

Best val_accuracy So Far: 0.8704883456230164
Total elapsed time: 07h 27m 38s

Search: Running Trial #84

Value             |Best Value So Far |Hyperparameter
0.2               |0.3               |Scale 1
0.1               |0.25              |Scale 2
0.2               |0.2               |Scale 3
50                |50                |tuner/epochs
17                |17                |tuner/initial_epoch
1                 |1                 |tuner/bracket
1                 |1                 |tuner/round
0075              |0080              |tuner/trial_id

Epoch 18/50
107/107 [==============================] - 68s 490ms/step - loss: 2.8456 - accuracy: 0.3027 - val_loss: 32.8960 - val_accuracy: 0.1338
Epoch 19/50
107/107 [==============================] - 49s 458ms/step - loss: 2.3015 - accuracy: 0.3545 - val_loss: 3.3551 - val_accuracy: 0.3461
Epoch 20/50
107/107 [==============================] - 47s 439ms/step - loss: 2.

In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
Scale 1: 0.1
Score: 0.37154990434646606
Trial summary
Hyperparameters:
Scale 1: 0.15
Score: 0.36305731534957886
Trial summary
Hyperparameters:
Scale 1: 0.3
Score: 0.36093416810035706
Trial summary
Hyperparameters:
Scale 1: 0.2
Score: 0.35881105065345764
Trial summary
Hyperparameters:
Scale 1: 0.25
Score: 0.31210190057754517


In [ ]:
#11:30
from numba import cuda
import gc

test = []
model = []
i, j, k = 0.1, 0.2, 0.15

# tf.keras.backend.clear_session()
# tf.keras.backend.set_learning_phase(True)
model = IRV2(X_train.shape[1:], 48, activation= 'softmax', scale1=i, scale2=j, scale3=k).build_model(kt.HyperParameters())
tuner = kt.RandomSearch(
    hypermodel=model,
    objective="val_accuracy"
)
tuner.search_space_summary()


TypeError: ignored

In [ ]:
reset_keras()

1880


In [ ]:
test_scales = [0.1, 0.15, 0.2, 0.25, 0.3]
test = []
for i in test_scales:
    for j in test_scales:
        for k in test_scales:
            if i != 0.1 or j != 0.1 or (k != 0.1 and k != 0.15):
                print(str(i) + ' ' + str(j) + ' ' + str(k))

0.1 0.1 0.2
0.1 0.1 0.25
0.1 0.1 0.3
0.1 0.15 0.1
0.1 0.15 0.15
0.1 0.15 0.2
0.1 0.15 0.25
0.1 0.15 0.3
0.1 0.2 0.1
0.1 0.2 0.15
0.1 0.2 0.2
0.1 0.2 0.25
0.1 0.2 0.3
0.1 0.25 0.1
0.1 0.25 0.15
0.1 0.25 0.2
0.1 0.25 0.25
0.1 0.25 0.3
0.1 0.3 0.1
0.1 0.3 0.15
0.1 0.3 0.2
0.1 0.3 0.25
0.1 0.3 0.3
0.15 0.1 0.1
0.15 0.1 0.15
0.15 0.1 0.2
0.15 0.1 0.25
0.15 0.1 0.3
0.15 0.15 0.1
0.15 0.15 0.15
0.15 0.15 0.2
0.15 0.15 0.25
0.15 0.15 0.3
0.15 0.2 0.1
0.15 0.2 0.15
0.15 0.2 0.2
0.15 0.2 0.25
0.15 0.2 0.3
0.15 0.25 0.1
0.15 0.25 0.15
0.15 0.25 0.2
0.15 0.25 0.25
0.15 0.25 0.3
0.15 0.3 0.1
0.15 0.3 0.15
0.15 0.3 0.2
0.15 0.3 0.25
0.15 0.3 0.3
0.2 0.1 0.1
0.2 0.1 0.15
0.2 0.1 0.2
0.2 0.1 0.25
0.2 0.1 0.3
0.2 0.15 0.1
0.2 0.15 0.15
0.2 0.15 0.2
0.2 0.15 0.25
0.2 0.15 0.3
0.2 0.2 0.1
0.2 0.2 0.15
0.2 0.2 0.2
0.2 0.2 0.25
0.2 0.2 0.3
0.2 0.25 0.1
0.2 0.25 0.15
0.2 0.25 0.2
0.2 0.25 0.25
0.2 0.25 0.3
0.2 0.3 0.1
0.2 0.3 0.15
0.2 0.3 0.2
0.2 0.3 0.25
0.2 0.3 0.3
0.25 0.1 0.1
0.25 0.1 0.15
0.25 0.1 0.2


In [ ]:
from tensorflow.keras.optimizers import Adam
model = IRV2(X_train.shape[1:], 48, activation= 'softmax').build_model()
model.compile(loss='sparse_categorical_crossentropy',optimizer="adam",
              metrics=['accuracy'])
epochs = 40
history = model.fit(X_train, y_train_e, batch_size=32, epochs=epochs, validation_data=(X_val, y_val_e))

Epoch 1/40
138/138 [==============================] - 101s 516ms/step - loss: 2.7708 - accuracy: 0.3179 - val_loss: 87.5530 - val_accuracy: 0.1353
Epoch 2/40
138/138 [==============================] - 64s 462ms/step - loss: 2.1505 - accuracy: 0.3938 - val_loss: 2.0749 - val_accuracy: 0.4099
Epoch 3/40
138/138 [==============================] - 64s 462ms/step - loss: 1.8565 - accuracy: 0.4675 - val_loss: 1.8978 - val_accuracy: 0.4793
Epoch 4/40
138/138 [==============================] - 64s 461ms/step - loss: 1.5847 - accuracy: 0.5312 - val_loss: 2.5118 - val_accuracy: 0.4024
Epoch 5/40
138/138 [==============================] - 64s 461ms/step - loss: 1.2351 - accuracy: 0.6320 - val_loss: 1.2971 - val_accuracy: 0.6071
Epoch 6/40
138/138 [==============================] - 64s 461ms/step - loss: 0.9691 - accuracy: 0.7050 - val_loss: 1.4337 - val_accuracy: 0.6003
Epoch 7/40
138/138 [==============================] - 64s 461ms/step - loss: 0.7225 - accuracy: 0.7779 - val_loss: 1.6697 - val_

In [ ]:
print(max(history.history['val_accuracy']))

0.8776342868804932


In [ ]:
from google.colab import files
import pandas as pd
model.save("IRv2")
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'history_IRv2_70.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

files.download('history_IRv2_70.json')
#files.download('IRv2')

INFO:tensorflow:Assets written to: IRv2/assets


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model = IRV2(X_train.shape[1:], 47, activation= 'softmax').build_model()
# decay = .9 e = 1.0 lr = 0.045, exp rate = 0.94
model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.optimizers.SGD(learning_rate=0.01),
              metrics=['accuracy'])


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, ZeroPadding2D, BatchNormalization
model=Sequential()

model.add(Conv2D(96,(11,11),strides=4,activation='relu',input_shape=X_train.shape[1:],padding='valid'))
model.add(MaxPool2D((3,3),strides=2))
model.add(BatchNormalization())

model.add(ZeroPadding2D(padding=2))
model.add(Conv2D(256,(5,5),activation='relu',padding='same'))
model.add(MaxPool2D((3,3), strides=2))
model.add(BatchNormalization())

model.add(ZeroPadding2D(padding=1))
model.add(Conv2D(384,(3,3),activation='relu',padding='same'))

model.add(ZeroPadding2D(padding=1))
model.add(Conv2D(384,(3,3),activation='relu',padding='same'))

model.add(ZeroPadding2D(padding=1))
model.add(Conv2D(256,(3,3),activation='relu',padding='same'))

model.add(Flatten())
model.add(Dense(4096,activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(4096,activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(48,activation="softmax"))

model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.optimizers.SGD(learning_rate=0.001),
              metrics=['accuracy'])
#callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.1, patience=5, verbose=1,
    mode='max', min_delta=0.0001, cooldown=2, min_lr=0)
epochs = 50
history = model.fit(X_train, y_train_e, batch_size=32, epochs=epochs,
validation_data=(X_val, y_val_e), callbacks = [callback])

Epoch 1/50
138/138 [==============================] - 31s 125ms/step - loss: 2.9496 - accuracy: 0.2343 - val_loss: 2.6825 - val_accuracy: 0.2726 - lr: 0.0010
Epoch 2/50
138/138 [==============================] - 13s 95ms/step - loss: 2.6389 - accuracy: 0.2853 - val_loss: 2.4061 - val_accuracy: 0.3623 - lr: 0.0010
Epoch 3/50
138/138 [==============================] - 13s 95ms/step - loss: 2.4183 - accuracy: 0.3435 - val_loss: 2.1711 - val_accuracy: 0.4310 - lr: 0.0010
Epoch 4/50
138/138 [==============================] - 13s 95ms/step - loss: 2.1904 - accuracy: 0.3923 - val_loss: 1.9650 - val_accuracy: 0.4602 - lr: 0.0010
Epoch 5/50
138/138 [==============================] - 13s 96ms/step - loss: 1.9691 - accuracy: 0.4521 - val_loss: 1.7649 - val_accuracy: 0.5337 - lr: 0.0010
Epoch 6/50
138/138 [==============================] - 13s 95ms/step - loss: 1.7344 - accuracy: 0.5037 - val_loss: 1.5537 - val_accuracy: 0.5812 - lr: 0.0010
Epoch 7/50
138/138 [==============================] - 13s